### BERT Practice
#### v. 20230820

In [ ]:
# 20230804 Pick up from here later :

# https://www.run.ai/guides/machine-learning-operations
# https://www.run.ai/guides/machine-learning-engineering/machine-learning-workflow
# https://www.run.ai/guides/machine-learning-engineering/machine-learning-automation

# https://research.google.com/colaboratory/local-runtimes.html
# https://stackoverflow.com/questions/60235353/docker-is-not-running-on-colab

"""
https://www.youtube.com/watch?v=0S81koZpwPA
install CUDA Toolkit, install WSL2, check for cuda availability!!
https://discuss.tensorflow.org/t/fail-to-use-4070ti/17447
Ubuntu vs WSL2 for Tensorflow??
"""


'\nhttps://www.youtube.com/watch?v=0S81koZpwPA\ninstall CUDA Toolkit, install WSL2, check for cuda availability!!\nhttps://discuss.tensorflow.org/t/fail-to-use-4070ti/17447\nUbuntu vs WSL2 for Tensorflow??\n'

In [ ]:
# pip3 install torch torchvision torchaudio     # for PyTorch without GPU, just CPU


In [ ]:
# Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`
# https://stackoverflow.com/questions/76448287/how-can-i-solve-importerror-using-the-trainer-with-pytorch-requires-accele
# Most probably you might have an ImportError at the first line if accelerate is not already installed when you installed transformers.

!pip install -U accelerate
!pip install transformers -U
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117    # install PyTorch

import accelerate
import transformers

transformers.__version__, accelerate.__version__

Looking in indexes: https://download.pytorch.org/whl/cu117


('4.27.4', '0.21.0')

In [ ]:
# Check if the GPU can be detected
# tensorflow-gpu has been removed. tensorflow package supports GPU accelerated operations via Nvidia CUDA.
import tensorflow as tf

device_name = tf.test.gpu_device_name()              # '/device:GPU:0' means GPU is enabled
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
import torch
# torch.cuda.is_available = lambda : False
""" # old way
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)
"""
# use my own GPU to train; move tensor to my GPU; https://www.run.ai/guides/gpu-deep-learning/pytorch-gpu
if torch.cuda.is_available():
   dev = "cuda:0"
else:
    dev = "cpu"
device = torch.device(dev)
print(device)
t1 = torch.zeros(4,3)
print(t1)
t1 = t1.to(device)
print(t1)

cuda:0
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], device='cuda:0')


In [ ]:
print(tf.config.list_physical_devices('GPU'))

# device_lib.list_local_devices()

torch.cuda.is_available()

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


True

In [ ]:
# https://medium.com/ai%C2%B3-theory-practice-business/use-gpu-in-your-pytorch-code-676a67faed09

# Make sure the same device is used for tensor allocation during all operations

a = t1.get_device()                    # returns the index of the GPU on which the tensor resides
b = torch.tensor(t1.shape).to(dev)     # use this index to direct placement for new tensors

# torch.cuda.set_device(  {GPU ID}  )     # alternative method to tensor placement

print(a)
print(b)

0
tensor([4, 3], device='cuda:0')


In [ ]:
# https://medium.com/ai%C2%B3-theory-practice-business/use-gpu-in-your-pytorch-code-676a67faed09

# Check running environment info regarding Cuda and devices

import sys
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION', )

from subprocess import call
# call(["nvcc", "--version"]) does not work
! nvcc --version
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')

# call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
print('Active CUDA Device: GPU', torch.cuda.current_device())
print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())


__Python VERSION: 3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
__pyTorch VERSION: 2.0.1+cu118
__CUDA VERSION
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
__CUDNN VERSION: 8700
__Number CUDA Devices: 1
__Devices
Active CUDA Device: GPU 0
Available devices  1
Current cuda device  0


In [ ]:

# And check device information of my graphics driver

!pip install pycuda
import pycuda.driver as cuda
cuda.init()
## Get Id of default device
torch.cuda.current_device()
# 0
cuda.Device(0).name()   # '0' is the id of my GPU

'NVIDIA GeForce RTX 4070 Ti'

In [ ]:
!cat /proc/meminfo            # check memory resources available

MemTotal:       16263492 kB
MemFree:         7511844 kB
MemAvailable:   10881616 kB
Buffers:          351380 kB
Cached:          2814768 kB
SwapCached:          100 kB
Active:          1841792 kB
Inactive:        5760428 kB
Active(anon):       7252 kB
Inactive(anon):  4510444 kB
Active(file):    1834540 kB
Inactive(file):  1249984 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:       4194304 kB
SwapFree:        4189660 kB
Dirty:               524 kB
Writeback:             0 kB
AnonPages:       4336768 kB
Mapped:          1347896 kB
Shmem:             85212 kB
KReclaimable:     617864 kB
Slab:             772960 kB
SReclaimable:     617864 kB
SUnreclaim:       155096 kB
KernelStack:       20864 kB
PageTables:        38968 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    12326048 kB
Committed_AS:   13276724 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       44740 kB
VmallocChunk:          0 kB
Percpu:          

In [ ]:
# install wandb for tracking data on dashboard
!pip install datasets wandb evaluate -qU
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/pytorch/text-classification/run_glue.py

--2023-08-16 22:24:40--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/pytorch/text-classification/run_glue.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28270 (28K) [text/plain]
Saving to: ‘run_glue.py.1’

run_glue.py.1       100%[===================>]  27.61K  --.-KB/s    in 0.004s  

2023-08-16 22:24:40 (6.13 MB/s) - ‘run_glue.py.1’ saved [28270/28270]



In [ ]:
# the run_glue.py script requires transformers dev
!pip install -q git+https://github.com/huggingface/transformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import wandb

# log in to have data synced to account
wandb.login()                         # uncomment to enter API key when this line runs

# log every trained model
%env WANDB_LOG_MODEL=true

wandb: Currently logged in as: ojw92. Use `wandb login --relogin` to force relogin


env: WANDB_LOG_MODEL=true


In [ ]:

# https://github.com/PradipNichite/Youtube-Tutorials/blob/main/FineTune_BERT_Model_Youtube.ipynb



import pandas as pd

# filter training data to desired dates (after 2022.2.22)
df = pd.read_csv('/test.csv', index_col=0).drop_duplicates()
#df = df[df.columns[0:5]]

# filter training data to desired dates
from datetime import datetime, timedelta

df = df[df.Date.apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S').date() > datetime(2022,2,21).date())]

# For class imbalance, use roughly same ratio of R & N
# BERT doesn't need class imbalance addressed
# df = pd.concat([df[df.Class=='R'], df[df.Class=='N'].iloc[::3, :]])

# Reduce data to reduce training time to test features
# df22 = df22.sample(frac=0.01, random_state=5)




In [ ]:
# Define Title + Content concatenator
from timeit import default_timer as timer
import re

def data_concat(df22, R_known=True):

    # Split the input dataframe into Text (Title + Contents) and Classes dataframes
    # input must have 3 columns of string entries (Title, Contents, and Classes)

    # check if every row entry of each column is string type (some are NaN, so no)
    # print(df22.applymap(lambda x : type(x).__name__).eq({'Title': 'str', 'Content': 'str', 'Class':'str'}))

    # convert NaN to empty strings (NaN -> str)
        # df22.apply(str) converts all columns to str, as well
    df22 = df22.replace(float('nan'), '', regex=True)

    # concatenate strings of title & content with a " " in between (1 body of text)
    df22['Text'] = df22['Title'] + " " + df22['Content']      # slicing DataFrame via .iloc[:,0] makes it a Series
    df22 = df22.loc[: , ['Text', 'Class']]    # so initialize it as a DataFrame. pd.DataFrame(some_Series) works

    if R_known == True:
    # R, r, YR = 1;     N, n, YN = 0
        R_cases = re.compile('R|YR', re.IGNORECASE)
        N_cases = re.compile('N|YN', re.IGNORECASE)
        df22['Class'] = df22['Class'].replace(to_replace=R_cases, value=1)
        df22['Class'] = df22['Class'].replace(to_replace=N_cases, value=0)
    else:
        # R_known == False; prepping not yet classified data
        Y_N_cases = re.compile('Y|N', re.IGNORECASE)
        df22['Class'] = df22['Class'].replace(to_replace=Y_N_cases, value=0)     # all N's for simplicity

    df22['Class'] = df22['Class'].astype('int32')


    return df22


In [ ]:
# combine title & content as text22, clean the text, then combine it with labels to a single df
df = data_concat(df)
df.head()

,Text,Class
452,Zfold 3 or S22 ultra Trying to decide between ...,0
453,S22 video cam Anyone tried out the video camer...,0
454,Thinking about trading my S21Ultra for S22+ Ev...,0
455,S21 Ultra vs S22+ Both phones are currently at...,0
456,"S21 or S22 Base/Standard Model Hey All,\n\nLoo...",0


In [ ]:
df['Class'].value_counts()

0    21624
1     7297
Name: Class, dtype: int64

In [ ]:
!pip uninstall tokenizers, transformers
!pip install transformers==4.27.4 -U

ERROR: Invalid requirement: 'tokenizers,'
  Using cached transformers-4.27.4-py3-none-any.whl (6.8 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.32.0.dev0
    Uninstalling transformers-4.32.0.dev0:
      Successfully uninstalled transformers-4.32.0.dev0


In [ ]:
# Note:

# Below error kept popping up when trying to run 'from transformers import TrainingArguments, Trainer' and 'from transformers import BertTokenizer, BertForSequenceClassification'
"""
RuntimeError: Failed to import transformers.training_args because of the following error (look up to see its traceback):
cannot import name 'is_torch_npu_available' from 'transformers.utils' (/usr/local/lib/python3.10/dist-packages/transformers/utils/__init__.py) site:stackoverflow.com
"""
# Found a work around by changing BertTokenizer into transformers.BertTokenizer and BertForSequenceClassification to transformers.BertForSequenceClassification

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
#from transformers import TrainingArguments, Trainer


#from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
model = transformers.BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# model = model.to( torch.device('cuda') )     # need NVIDIA driver for 'cuda'; currently have AMD on work laptop
# model = model.to('cpu')         # train on CPU

model = model.to('cuda')          # or  model.cuda()


In [ ]:

test_data = ["This is possibly the worst battery I have ever seen on a mobile device",
            "How is my device running so smoothly?"]
tokenizer(test_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 2023, 2003, 4298, 1996, 5409, 6046, 1045, 2031, 2412, 2464, 2006, 1037, 4684, 5080, 102], [101, 2129, 2003, 2026, 5080, 2770, 2061, 15299, 1029, 102, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]}

In [ ]:
X = list(df["Text"])
y = list(df["Class"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [ ]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
print(X_train_tokenized['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
len(X_train),len(X_val)


(23136, 5785)

In [ ]:
# Create torch dataset
class VOC_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
train_dataset = VOC_Dataset(X_train_tokenized, y_train)
val_dataset = VOC_Dataset(X_val_tokenized, y_val)

In [ ]:
train_dataset[5]


{'input_ids': tensor([  101,  5292, 20746, 12247,  1998, 26828, 17880,  7632,  3071,  1010,
          2074,  2363,  2026,  1055, 19317, 11087,  1998,  9107,  2009,  2005,
          1996,  2087,  2112,  2021,  1045,  3342,  2070,  2111,  3331,  2055,
          1037,  8276,  2124,  5056,  2001,  1996,  5292, 20746, 12247,  1998,
         26828, 17880,  2001,  4011,  2000,  2022,  1037,  2843,  2488,  2013,
          2026,  3325,  2009,  2003,  1996,  2168,  2012,  2190,  3087,  2842,
          5060,  2023,  1010,  2001,  2023,  7620,  2000,  2023,  3444,  4704,
          2030,  2572,  1045,  2074,  2025,  3773,  2009,  2006,  1996, 10906,
          1029,  1045,  2031,  2673,  2012,  2440, 17880,  1998,  1045, 10657,
          2053,  4489,  2012,  2035,   102,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [ ]:
def compute_metrics(m):
    print(type(m))
    pred, labels = m
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


In [ ]:
# Define Trainer
args = transformers.TrainingArguments(       # from transformers import TrainingArguments spat our error so do transformers.TrainingArguments
    report_to = 'wandb',                     # enable logging to W&B
    output_dir="output",                     # output directory
    num_train_epochs=4,
    per_device_train_batch_size=8
    # overwrite_output_dir = True,
    # evaluation_strategy = 'steps',          # check evaluation metrics at each epoch
    # learning_rate = 5e-5,                   # we can customize learning rate
    # max_steps = 30000,
    # logging_steps = 100,                    # we will log every 100 steps
    # eval_steps = 5000,                      # we will perform evaluation every 500 steps
    # save_steps = 10000,
    # load_best_model_at_end = True,
    # metric_for_best_model = 'accuracy',
    # run_name = 'custom_training'            # name of the W&B run

)
trainer = transformers.Trainer(               # from transformers import Trainer spat our error so do transformers.Trainer
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,                 # for padding batched data
    compute_metrics=compute_metrics
)

In [ ]:
# https://github.com/facebookresearch/fairseq/issues/1933
# https://huggingface.co/spaces/OFA-Sys/OFA-Generic_Interface/blob/8fc1d8aafce5821301443744696303bac6227f52/fairseq/examples/roberta/commonsense_qa/README.md

# PyTorch fairseq

MAX_UPDATES=2300      # Number of training steps, where   training step = training set / batch size. Originally 3000
WARMUP_UPDATES=150    # Linearly increase LR over this many steps
MAX_EPOCH=4           # Number of training epochs.
LR=1e-05              # Peak LR for fixed LR scheduler. 1e-05 default; try 1e-06 as initial LR
NUM_CLASSES=2
MAX_SENTENCES=8       # Batch size per GPU.
UPDATE_FREQ=32        # Accumulate gradients to simulate training on 8 GPUs.
DATA_DIR='VOC_final_output'
ROBERTA_PATH='/SiERoBERT_large/model.pt'

! CUDA_VISIBLE_DEVICES=0 fairseq-train $DATA_DIR --ddp-backend=no_c10d \
  --restore-file $ROBERTA_PATH \
  --reset-optimizer --reset-dataloader --reset-meters \
  --no-epoch-checkpoints --no-last-checkpoints --no-save-optimizer-state \
  #--best-checkpoint-metric accuracy --maximize-best-checkpoint-metric \
  --task sentence_ranking \
  --num-classes $NUM_CLASSES \
  --init-token 0 --separator-token 2 \
  --max-option-length 128 \
  --max-positions 512 \
  --truncate-sequence \
  --arch roberta_large \
  --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
  --criterion sentence_ranking \
  --optimizer adam --adam-betas '(0.9, 0.98)' --adam-eps 1e-06 \
  --clip-norm 0.0 \
  --lr-scheduler fixed --lr $LR \
  --warmup-updates $WARMUP_UPDATES --total-num-update $MAX_UPDATES \
  --memory-efficient-fp16 --fp16-init-scale 4 --threshold-loss-scale 1 --fp16-scale-window 128 \
  --batch-size $MAX_SENTENCES \
  --required-batch-size-multiple 1 \
  --update-freq $UPDATE_FREQ \
  --max-epoch $MAX_EPOCH \
  --log-interval 100 \

# --memory-efficient-fp16 instead of --fp16 solves 'CUDA out of memory' problem, but slow training
# --max-sentences $MAX_SENTENCES , not batch-size
# --fp16-scale-window 128 default

/bin/bash: line 1: fairseq-train: command not found


In [ ]:
# clear cache before training
torch.cuda.empty_cache()

In [ ]:
from timeit import default_timer as timer
start = timer()

trainer.train()


end = timer()
print("%4f seconds, %4f minutes elapsed" % (float(end-start), float((end-start)/60)))

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.443500
1000,0.383600
1500,0.397800
2000,0.361200
2500,0.419400
3000,0.385500
3500,0.338800
4000,0.338500
4500,0.340300
5000,0.317700


2434.823252 seconds, 40.580388 minutes elapsed


In [ ]:
trainer.evaluate()


<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.4116346836090088,
 'eval_accuracy': 0.8936905790838375,
 'eval_precision': 0.78839590443686,
 'eval_recall': 0.791095890410959,
 'eval_f1': 0.7897435897435896,
 'eval_runtime': 244.6073,
 'eval_samples_per_second': 23.65,
 'eval_steps_per_second': 2.96,
 'epoch': 4.0}

In [ ]:
np.set_printoptions(suppress=True)


In [ ]:
text = "Super charging is working very well."
# text = "so many issues with this phone."
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)    # also gelu(), silu()
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5494,  1.2647]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.0566, 0.9434]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


array([[0.05656639, 0.94343364]], dtype=float32)

In [ ]:
trainer.save_model('BertPractice')

In [ ]:
# trainer.save_model('/content/drive/MyDrive/Youtube Tutorials/toxic')
# model_2 = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Youtube Tutorials/toxic")
# model_2.to('cuda')

In [ ]:
model_2 = transformers.BertForSequenceClassification.from_pretrained('BertPractice')
model_2.to('cuda')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# text = "Super charging is working very well."
text = "so many issues with this phone."
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions

array([[0.04935925, 0.95064074]], dtype=float32)

In [ ]:
# Make predictions on daily data

def pred_day(voctoday):
  voclist = list(voctoday['Text'])
  predlist = []

  for i in range(len(voclist)):
    inputs = tokenizer(voclist[i], padding = True, truncation = True, return_tensors='pt').to('cuda')
    outputs = model_2(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predictions = predictions.cpu().detach().numpy()
    predlist.append(predictions)

  organizedzip = zip([x[:50] for x in voclist],
                    predlist,
                    voctoday['Class'])

  # results as voc text, predictions, and actual value
  vocresult = pd.DataFrame(list(organizedzip), columns=['Text','Prediction','Actual'])

  return vocresult


In [ ]:

# test VOC
# import file and combine title & content

#voc0123 = pd.read_csv('/20230123_test.csv', index_col=0).drop_duplicates()
#voc0123 = data_concat(voc0123.loc[: , 'Title':'Class'])

voc801 = pd.read_csv('/801testraw.csv', index_col=0, delimiter="\t").drop_duplicates()




############## delete this in-between chunk later~
R_cases = re.compile('R|YR', re.IGNORECASE)
N_cases = re.compile('N|YN', re.IGNORECASE)
voc801['Class'] = voc801['Class'].replace(to_replace=R_cases, value=1)
voc801['Class'] = voc801['Class'].replace(to_replace=N_cases, value=0)

# R_known == False; prepping not yet classified data
Y_N_cases = re.compile('Y|N', re.IGNORECASE)
voc801['Class'] = voc801['Class'].replace(to_replace=Y_N_cases, value=0)     # all N's for simplicity

voc801['Class'] = voc801['Class'].astype('int32')
################# delete above~





voc801 = data_concat(voc801.loc[: , 'Title':'Class'])

voc802 = pd.read_csv('/802testraw.csv', index_col=0, delimiter="\t").drop_duplicates()




############ delete this in-between chunk later~
R_cases = re.compile('R|YR', re.IGNORECASE)
N_cases = re.compile('N|YN', re.IGNORECASE)
voc802['Class'] = voc802['Class'].replace(to_replace=R_cases, value=1)
voc802['Class'] = voc802['Class'].replace(to_replace=N_cases, value=0)

# R_known == False; prepping not yet classified data
Y_N_cases = re.compile('Y|N', re.IGNORECASE)
voc802['Class'] = voc802['Class'].replace(to_replace=Y_N_cases, value=0)     # all N's for simplicity

voc802['Class'] = voc802['Class'].astype('int32')
############# delete above~





voc802 = data_concat(voc802.loc[: , 'Title':'Class'])

#vocresult0123 = pred_day(voc0123)
#vocresult0123

ValueError: ignored

In [ ]:
vocresult801 = pred_day(voc801)
vocresult801

,Text,Prediction,Actual
0,Is S22 Ultra good at this year ? I'm planning ...,"[[0.9983941, 0.0016059423]]",0
1,Case for s22u Do you recommend I use a case fo...,"[[0.99846673, 0.0015333229]]",0
2,Speak to a uk based/us based operator samsung ...,"[[0.9984577, 0.0015422924]]",0
3,So Exynos is confirmed to come back for s24 se...,"[[0.9984762, 0.0015237588]]",0
4,Why can't the camera shoot 27 megapixels? The ...,"[[0.9984303, 0.0015696192]]",0
5,Webview keeps crashing Samsung Health after Ju...,"[[0.024444215, 0.9755558]]",0
6,"Change the ""scroll behavior"" of the notificati...","[[0.92239755, 0.077602506]]",0
7,GCam for S22 Are any of you using GCam in your...,"[[0.99573755, 0.004262493]]",0
8,Unable to show details of notifications on the...,"[[0.025129264, 0.9748708]]",0
9,Should I upgrade my s22 for an s22+ Title,"[[0.99840087, 0.001599121]]",0


In [ ]:
vocresult802 = pred_day(voc802)
vocresult802

,Text,Prediction,Actual
0,Possible battery and over,"[[0.075512774, 0.9244873]]",0
1,Turning this setting off,"[[0.9628211, 0.037178855]]",0
2,Call Problem Does anyone,"[[0.02485165, 0.9751483]]",0
3,Stops charging at random,"[[0.024824362, 0.9751756]]",0
4,Losing service and pingin,"[[0.023929827, 0.9760702]]",0
5,I have a samsung galaxy s,"[[0.9984187, 0.0015812749]]",0
6,S22 Ultra display twirks?,"[[0.9941858, 0.0058142045]]",0
7,Would turning off always,"[[0.99832195, 0.0016780444]]",0
8,Wireless charger charging,"[[0.024863997, 0.9751359]]",0
9,Will s22 ultra recieve on,"[[0.99847335, 0.0015266751]]",0


In [ ]:

# test VOC for 8/14 (Mon) - 8/16 (Wed) using model trained on ~8/11 (Fri) data

voc814 = pd.read_csv('/814testraw.csv', index_col=0, delimiter=",").drop_duplicates()     # use delimiter ',' for original .csv file created from my test file generator
voc814 = data_concat(voc814.loc[: , 'Title':'Class'])

voc815 = pd.read_csv('/815testraw.csv', index_col=0, delimiter=",").drop_duplicates()
voc815 = data_concat(voc815.loc[: , 'Title':'Class'])

voc816 = pd.read_csv('/816testraw.csv', index_col=0, delimiter=",").drop_duplicates()
voc816 = data_concat(voc816.loc[: , 'Title':'Class'])

In [ ]:
vocresult814 = pred_day(voc814)
vocresult814

,Text,Prediction,Actual
0,Please help s22 vs S23. Should I pay the extra...,"[[0.99836224, 0.0016377212]]",0
1,Trade in question - screen issue My s22 ultra ...,"[[0.99381846, 0.006181567]]",1
2,Routine How could I change the mode on my phon...,"[[0.99795, 0.0020499814]]",0
3,S22 ultra root. I've been trying to root my s2...,"[[0.99682343, 0.0031765667]]",0
4,S22Ultra Cases and Whitestone FOR SALE\n\nSwit...,"[[0.99842227, 0.0015777047]]",0
5,Fast Charging suddenly stopped working &amp; c...,"[[0.024860265, 0.9751397]]",0
6,"Super Fast Charging 2.0 Hi, I have an S22 Ultr...","[[0.972452, 0.027548041]]",0
7,Broken Screen. Options? Cracked my screen wee...,"[[0.06370803, 0.936292]]",1
8,S22u green bar on camera issue is happening. I...,"[[0.036256384, 0.9637437]]",1
9,"Planing To Move To IOS Hello everyone, I'd lik...","[[0.99819666, 0.00180335]]",0


In [ ]:
vocresult815 = pred_day(voc815)
vocresult815

,Text,Prediction,Actual
0,Currently using s22 ultra and thinking of swit...,"[[0.9984072, 0.0015928753]]",0
1,Any reason why is S23 series behind S22 in upd...,"[[0.9981421, 0.0018578192]]",0
2,Routine not running at specified time I have a...,"[[0.02439786, 0.97560215]]",1
3,Wifi issue on s22 ultra My wifi randomly disco...,"[[0.02395671, 0.9760433]]",1
4,Camera lag Generally I feel like the camera ap...,"[[0.024440384, 0.97555965]]",1
5,Phones unable to keep up. I have started notic...,"[[0.025178522, 0.9748215]]",1
6,Screen issue (blinking pixel?) As seen in the ...,"[[0.025062656, 0.9749374]]",1
7,Looking For Good Grip Slim Case Any Recommende...,"[[0.99844897, 0.0015510272]]",0
8,Change microphone source input Hi\n\nWhen I'm ...,"[[0.99683195, 0.003167996]]",0
9,Recent WiFi Troubles Have a bit of a weird iss...,"[[0.02451749, 0.97548246]]",1


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e5caba9f-cd36-4d50-aaa3-2cf59957a2f4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>